In [1]:
import pandas as pd
print("Hello")
df = pd.read_csv("./data.csv", index_col="index")
print(df.head())

Hello
              host       time method                           url  response  \
index                                                                          
0      ***.novo.dk  805465029    GET                     /ksc.html       200   
1      ***.novo.dk  805465031    GET    /images/ksclogo-medium.gif       200   
2      ***.novo.dk  805465051    GET  /images/MOSAIC-logosmall.gif       200   
3      ***.novo.dk  805465053    GET     /images/USA-logosmall.gif       200   
4      ***.novo.dk  805465054    GET    /images/NASA-logosmall.gif       200   

       bytes  
index         
0       7067  
1       5866  
2        363  
3        234  
4        786  


In [2]:
# removing all not needed GET requests
remove_media = df[
    df.url.str.contains(".html", case=False) &
    df.url.str.contains(".htm", case=False) |
    df.url.str.endswith("/") 
]

In [8]:
grouped_by_site = remove_media.groupby(by=["host"])
grouped_by_site.groups

dict_keys(['***.novo.dk', '001.msy4.communique.net', '007.thegap.com', '01-dynamic-c.wokingham.luna.net', '01.ts01.zircon.net.au', '02-17-05.comsvc.calpoly.edu', '02-dynamic-c.wokingham.luna.net', '023.msy4.communique.net', '03-dynamic-c.wokingham.luna.net', '0321jona.jon.rpslmc.edu', '033.msy4.communique.net', '04-dynamic-c.rotterdam.luna.net', '04-dynamic-c.wokingham.luna.net', '049.215.goodnet.com', '05-dynamic-c.rotterdam.luna.net', '05-dynamic-c.wokingham.luna.net', '052.215.goodnet.com', '06-dynamic-c.rotterdam.luna.net', '07-dynamic-c.rotterdam.luna.net', '07mb369b.uni-duisburg.de', '08-dynamic-c.rotterdam.luna.net', '086.msy4.communique.net', '0875pr3e.pro.rpslmc.edu', '0885-12.etech.fh-hamburg.de', '1.ts1.mnet.medstroms.se', '1.ts2.mnet.medstroms.se', '10-dynamic-c.rotterdam.luna.net', '10.salc.wsu.edu', '10.ts2.mnet.medstroms.se', '100.144.med.umich.edu', '10044.geo.net', '101.irri.cgiar.org', '103.135.med.umich.edu', '1032015.ksc.nasa.gov', '1032ap.sscl.uwo.ca', '1032by.sscl